In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np
def normalizedata(X_train):
    scaler = MinMaxScaler()
    scaler.fit(X_train)
    X_train = scaler.transform(X_train)
    return X_train[:,~np.any(np.isnan(X_train), axis=0)]

In [2]:
def allinput():
  inpf=[]
  fname='02_new_ranksums_wiki200.csv'
  data=np.genfromtxt(fname,delimiter=',')
  inpf.append(normalizedata(data[:,0:105]))
  inpf.append(normalizedata(data[:,105:-1]))
  lab=np.array(data[:,-1])
  inpf.append(lab)
  return inpf

In [3]:
inval=[]
a=[0,1]
inval.append(a)

In [4]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
XGB = XGBClassifier(random_state=13)

In [5]:
def final_model(trdata):
  model=[]
  model.append(MultinomialNB())
  model.append(BernoulliNB())
  model.append(GaussianNB())
  model.append(DecisionTreeClassifier())
  model.append(LogisticRegression())
  model.append(KNeighborsClassifier())
  model.append(svm.SVC(kernel="linear",probability=True))
  model.append(svm.SVC(kernel="poly",probability=True))
  model.append(svm.SVC(kernel="rbf",probability=True))
  model.append(BaggingClassifier(MultinomialNB(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(DecisionTreeClassifier(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(LogisticRegression(),max_samples=0.5, max_features=0.5))
  model.append(BaggingClassifier(KNeighborsClassifier(),max_samples=0.5, max_features=0.5))
  model.append(RandomForestClassifier(n_estimators=10))
  model.append(ExtraTreesClassifier(n_estimators=10, max_depth=None,min_samples_split=2, random_state=0))
  model.append(AdaBoostClassifier(n_estimators=10))
  model.append(GradientBoostingClassifier(n_estimators=10, learning_rate=1.0,max_depth=1, random_state=0))
  model.append(XGBClassifier())
  model.append(LGBMClassifier(metric='auc',verbose=-1))
  model.append(MLPClassifier(solver='lbfgs', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  model.append(MLPClassifier(solver='sgd', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  model.append(MLPClassifier(solver='adam', hidden_layer_sizes=(trdata.shape[1], 2), random_state=1,max_iter=1500))
  return model

In [6]:
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
def modelst2(m,X,a):
  clf_1 = m[0]
  clf_1_transformer = FunctionTransformer(lambda X: X[:,:a])
  tclf_1 = Pipeline([('transformer_1', clf_1_transformer), ('clf_1', clf_1)])
  clf_2 = m[1]
  clf_2_transformer = FunctionTransformer(lambda X: X[:,a:])
  tclf_2 = Pipeline([('transformer_2', clf_2_transformer), ('clf_2', clf_2)])
  sclf = StackingClassifier([('tclf_1', tclf_1), ('tclf_2', tclf_2)],final_estimator=XGB,cv=3)
  return sclf

In [7]:
from sklearn.datasets import make_classification
from sklearn.ensemble import StackingClassifier
from sklearn.preprocessing import FunctionTransformer
from sklearn.pipeline import Pipeline
def modelst2(m,X,a):
  clf_1 = m[0]
  clf_1_transformer = FunctionTransformer(lambda X: X[:,:a])
  tclf_1 = Pipeline([('transformer_1', clf_1_transformer), ('clf_1', clf_1)])
  clf_2 = m[1]
  clf_2_transformer = FunctionTransformer(lambda X: X[:,a:])
  tclf_2 = Pipeline([('transformer_2', clf_2_transformer), ('clf_2', clf_2)])
  sclf = StackingClassifier([('tclf_1', tclf_1), ('tclf_2', tclf_2)],final_estimator=XGB,cv=3)
  return sclf

def modelst3(m,X,a,b):
  clf_1 = m[0]
  clf_1_transformer = FunctionTransformer(lambda X: X[:,:a])
  tclf_1 = Pipeline([('transformer_1', clf_1_transformer), ('clf_1', clf_1)])
  clf_2 = m[1]
  clf_2_transformer = FunctionTransformer(lambda X: X[:,a:b])
  tclf_2 = Pipeline([('transformer_2', clf_2_transformer), ('clf_2', clf_2)])
  clf_3 = m[2]
  clf_3_transformer = FunctionTransformer(lambda X: X[:,b:])
  tclf_3 = Pipeline([('transformer_2', clf_3_transformer), ('clf_3', clf_3)])
  sclf = StackingClassifier([('tclf_1', tclf_1), ('tclf_2', tclf_2), ('tclf_3', tclf_3)],final_estimator=XGB,cv=3)
  return sclf

In [8]:
def final_model2(x,a):
  model1=final_model(x[:,:a])
  model2=final_model(x[:,a:])
  model3=final_model(x[:,0:2])
  model=[]
  for i in range(0,22):
    m=[]
    m.append(model1[i])
    m.append(model2[i])
    m.append(model3[i])
    model.append(modelst2(m,x,a))
  return model

In [9]:
def tr_model(model,trdata,tract,tsdata):
  pred=np.zeros((np.shape(tsdata)[0],22))
  pred1=np.zeros((np.shape(tsdata)[0],22))
  for i in range(0,22):
    m=model[i]
    m.fit(trdata,tract)
    pred[:,i]=m.predict(tsdata)
    pred1[:,i]=m.predict_proba(tsdata)[:,1]
  return pred,pred1

In [10]:
from random import sample
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
kf = KFold(3)
def final_tr_model(data,bugs,cs,a,b):
  predv=np.zeros((np.shape(data)[0],23))
  predv1=np.zeros((np.shape(data)[0],23))
  model=final_model2(data,a)
  list1 = [i for i in range(np.shape(data)[0])]
  in1=sample(list1,np.shape(data)[0])
  data=data[in1,:]
  bugs=bugs[in1]
  for train_index, test_index in kf.split(data):
    trainX=data[train_index,:]
    testX=data[test_index,:]
    tract=bugs[train_index]
    tsact=bugs[test_index]
    pred,pred1=tr_model(model,trainX,tract,testX)
    predv[test_index,0:22]=pred
    predv[test_index,22]=tsact
    predv1[test_index,0:22]=pred1
    predv1[test_index,22]=tsact
  return predv,predv1

In [11]:
for p in range(0,1):
  inpf=allinput()
  nor=np.shape(inpf[0])[0]
  predv=np.zeros((nor,23))
  predv1=np.zeros((nor,23))
  for i in range(0,len(inval)):
    print(i)
    if len(inval[i])==2:
      x=[]
      x.append(inpf[inval[i][0]])
      x.append(inpf[inval[i][1]])
      x1=np.concatenate((x[0], x[1]), axis=1)
      predv,predv1=final_tr_model(x1,inpf[-1],1,np.shape(x[0])[1],0)
    fname='pred'+str(p)+'_'+str(i)+'_pred.csv'
    np.savetxt(fname,predv, delimiter=',', fmt='%f')
    fname='pred'+str(p)+'_'+str(i)+'_predp.csv'
    np.savetxt(fname,predv1, delimiter=',', fmt='%f')

0


C:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result

In [12]:
inpf=allinput()
np.max(inpf[2])

1.0

In [13]:
from sklearn.metrics import (
    f1_score, precision_score, recall_score,accuracy_score
    )
import pandas as pd
import numpy as np
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import (
    f1_score, precision_score, recall_score,accuracy_score
    )
aucv=np.zeros((1,22))
for k in range(0,1):
  for i in range(0,1):
    fname='pred'+str(k)+'_'+str(i)+'_pred.csv'
    data=np.genfromtxt(fname,delimiter=',')
    fname='pred'+str(k)+'_'+str(i)+'_predp.csv'
    data1=np.genfromtxt(fname,delimiter=',')
    print('---------------',i)
    for j in range(0,22):
      fpr, tpr, _ = roc_curve(data1[:,-1],data1[:,j])
      aucv[i,j]=auc(fpr, tpr)
      print(accuracy_score(data[:,j],data[:,-1])*100,auc(fpr, tpr))

--------------- 0
67.4 0.734328169215102
65.64999999999999 0.6614978983322942
66.05 0.7195349396219896
66.75 0.6730407059285594
68.85 0.7553748930923434
71.1 0.7783794891826085
67.10000000000001 0.7421258553953892
66.45 0.7372169562786585
70.7 0.7853653746892031
66.05 0.7165978630272813
68.30000000000001 0.7492399764347087
67.4 0.7458935119259045
68.7 0.7622996118594796
70.1 0.7624772100240482
69.3 0.764407181619073
64.64999999999999 0.7213189695228672
65.45 0.7160870670977668
73.2 0.7977248119122762
68.8 0.7739153903028183
64.5 0.7165833761679359
67.95 0.7512289685678075
67.65 0.7478309952257749
